In [2]:
import requests, base64

invoke_url = "https://ai.api.nvidia.com/v1/vlm/nvidia/neva-22b"
stream = True

with open("path_to_image", "rb") as f:
    image_b64 = base64.b64encode(f.read()).decode()

assert len(image_b64) < 180_000, \
  "To upload larger images, use the assets API (see docs)"

headers = {
  "Authorization": "NVIDIA_API_KEY",
  "Accept": "text/event-stream" if stream else "application/json"
}

payload = {
  "messages": [
    {
      "role": "user",
      "content": f'Describe what you see in this image. <img src="data:image/jpg;base64,{image_b64}" />'
    }
  ],
  "max_tokens": 1024,
  "temperature": 0.20,
  "top_p": 0.70,
  "seed": 0,
  "stream": stream
}

response = requests.post(invoke_url, headers=headers, json=payload)

if stream:
    for line in response.iter_lines():
        if line:
            print(line.decode("utf-8"))
else:
    print(response.json())


AssertionError: To upload larger images, use the assets API (see docs)

In [16]:
from langchain_ollama import ChatOllama
# Load environment variables from .env
#load_dotenv()

# Create a ChatOpenAI model
#model = ChatOpenAI(model="gpt-4o")
model = ChatOllama(model = "llama3.1:8b", temperature = 0.3,)

# Invoke the model with a message
result = model.invoke("What is 81 divided by 9?")
print("Full result:")
print(result)
print("Content only:")
print(result.content)


Full result:
content='81 divided by 9 is:\n\n81 ÷ 9 = 9' additional_kwargs={} response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-01-17T06:18:39.3634088Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12622515600, 'load_duration': 7731257400, 'prompt_eval_count': 19, 'prompt_eval_duration': 2013000000, 'eval_count': 16, 'eval_duration': 2876000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-ddb0d785-c74b-46e5-8826-ebda0fca7df9-0' usage_metadata={'input_tokens': 19, 'output_tokens': 16, 'total_tokens': 35}
Content only:
81 divided by 9 is:

81 ÷ 9 = 9


In [6]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
model = ChatOllama(model = "llama3.1:8b", temperature = 0.4,)
messages = [
    SystemMessage(content="Translate the following question into Spanish. "),
    HumanMessage(content="What is 81 divided by 9?"),
]

# Invoke the model with messages
result = model.invoke(messages)
print(f"Answer from AI: {result.content}")
# chat_history.append(system_message)

# # AIMessage:
# #   Message from an AI.
# messages = [
#     SystemMessage(content="Solve the following math problems"),
#     HumanMessage(content="What is 81 divided by 9?"),
#     AIMessage(content="81 divided by 9 is 9."),
#     HumanMessage(content="What is 10 times 5?"),
# ]

# # Invoke the model with messages
# result = model.invoke(messages)
# print(f"Answer from AI: {result.content}")


Answer from AI: ¿Cuánto es 81 dividido por 9?


In [9]:
model = ChatOllama(model = "llama3.1:8b", temperature = 0.7, num_predict = 256,)
chat_history = []
system_message = SystemMessage(content = "You are very great at conversation and helpful.")
chat_history.append(system_message.content)

while True:
    query = input("You: ")
    if (query.lower()=="exit"):
        print("Chatbot is exiting.....")
        break
    chat_history.append(HumanMessage(content=query))
    result = model.invoke(chat_history)
    print(f"AI: {result.content}")
    chat_history.append(AIMessage(content=result.content))

print("------------Chat History----------")
print(chat_history)
    

You:  hola


AI: Hola! It's lovely to meet you. How's your day going so far? Is there something on your mind that you'd like to chat about or is it just a pleasure to say hola?


You:  what are trends in Manufacturing sector?


AI: The manufacturing sector is constantly evolving, and I'm happy to share some of the current trends with you.

**1. Industry 4.0**: Also known as the Fourth Industrial Revolution (4IR), this trend focuses on the integration of advanced technologies like AI, IoT, robotics, and data analytics to create smart factories that are more efficient, flexible, and connected.

**2. Digital Transformation**: Manufacturers are adopting digital tools and platforms to streamline operations, improve supply chain management, and enhance product development. This includes the use of cloud computing, big data analytics, and cybersecurity measures.

**3. Sustainability and Circular Economy**: Companies are shifting towards sustainable practices, reducing waste, and implementing circular economy models that minimize environmental impact while promoting resource efficiency.

**4. Automation and Robotics**: With advancements in robotics and automation technologies, manufacturers are incorporating these so

You:  what are your thoughts on Digital transformation. Give me pros and cons


AI: Digital transformation! It's a fascinating topic that's changing the way businesses operate. As a conversational AI, I've had the privilege of exploring various aspects of digital transformation with many organizations.

**Pros:**

1. **Increased Efficiency**: Digital transformation enables companies to automate manual tasks, streamline processes, and enhance productivity.
2. **Improved Customer Experience**: By leveraging data analytics and cloud-based services, businesses can create personalized experiences for their customers, leading to higher satisfaction rates and loyalty.
3. **Enhanced Innovation**: Digital technologies like AI, blockchain, and the Internet of Things (IoT) foster innovation, allowing companies to develop new products, services, and business models.
4. **Cost Savings**: By reducing manual labor, minimizing errors, and optimizing resource utilization, digital transformation can lead to significant cost savings for organizations.
5. **Competitive Advantage**: C

You:  exit


Chatbot is exiting.....
------------Chat History----------
['You are very great at conversation and helpful.', HumanMessage(content='hola', additional_kwargs={}, response_metadata={}), AIMessage(content="Hola! It's lovely to meet you. How's your day going so far? Is there something on your mind that you'd like to chat about or is it just a pleasure to say hola?", additional_kwargs={}, response_metadata={}), HumanMessage(content='what are trends in Manufacturing sector?', additional_kwargs={}, response_metadata={}), AIMessage(content="The manufacturing sector is constantly evolving, and I'm happy to share some of the current trends with you.\n\n**1. Industry 4.0**: Also known as the Fourth Industrial Revolution (4IR), this trend focuses on the integration of advanced technologies like AI, IoT, robotics, and data analytics to create smart factories that are more efficient, flexible, and connected.\n\n**2. Digital Transformation**: Manufacturers are adopting digital tools and platforms to

In [11]:
from langchain.prompts import ChatPromptTemplate

template = "Tell me a joke about {topic}."
prompt_template = ChatPromptTemplate.from_template(template)

print("-----Prompt from Template-----")
prompt = prompt_template.invoke({"topic": "cats"})
print(prompt)

-----Prompt from Template-----
messages=[HumanMessage(content='Tell me a joke about cats.', additional_kwargs={}, response_metadata={})]


In [12]:
template_multiple = """You are a helpful assistant.
Human: Tell me a {adjective} story about a {animal}.
Assistant:"""
prompt_multiple = ChatPromptTemplate.from_template(template_multiple)
prompt = prompt_multiple.invoke({"adjective": "funny", "animal": "panda"})
print("\n----- Prompt with Multiple Placeholders -----\n")
print(prompt)


----- Prompt with Multiple Placeholders -----

messages=[HumanMessage(content='You are a helpful assistant.\nHuman: Tell me a funny story about a panda.\nAssistant:', additional_kwargs={}, response_metadata={})]


In [13]:
messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    ("human", "Tell me {joke_count} jokes."),
]
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"topic": "lawyers", "joke_count": 3})
print("\n----- Prompt with System and Human Messages (Tuple) -----\n")
print(prompt)



----- Prompt with System and Human Messages (Tuple) -----

messages=[SystemMessage(content='You are a comedian who tells jokes about lawyers.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me 3 jokes.', additional_kwargs={}, response_metadata={})]
